In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
def search_baidu(keyword):
    """从百度搜索引擎中搜索关键词"""
    return f"{keyword}是一个技术博主"

def search_google(keyword):
    """从谷歌搜索引擎中搜索关键词"""
    return f"{keyword}是一个后端工程师"

def search_bing(keyword):
    """从必应搜索引擎中搜索关键词"""
    return f"{keyword}是一个Python爱好者"

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_baidu",
            "description": "从百度搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    },    
    {
        "type": "function",
        "function": {
            "name": "search_google",
            "description": "从google搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    },        
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "从bing搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    }
]
available_functions = { "search_baidu": search_baidu, "search_google": search_google, "search_bing": search_bing } 

In [12]:
from openai import OpenAI
import json
client = OpenAI(base_url='https://thales.xindoo.xyz/openai/v1/')

def search(keyword):
    messages = [{"role": "user", "content": f"汇总下百度、谷歌、必应三个搜索引擎关于'{keyword}'的结果"}]
    # 发起首次请求，告诉gpt要做什么，已经有哪些函数可以调动 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto", 
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # 检查是否需要调用函数
    if tool_calls:
        # 解析所有需要调用的函数及参数
        messages.append(response_message)  # 注意这里要将openai的回复也拼接到消息列表里
        # 将所有函数调用的结果拼接到消息列表里
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  
        return second_response.choices[0].message.content
    
print(search("xindoo"))

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E9tvxYXgM3aYhnMMjF5AbgeW', function=Function(arguments='{"keyword": "xindoo"}', name='search_baidu'), type='function'), ChatCompletionMessageToolCall(id='call_Z9Aw9mIzXKbrbF69LXwATBgT', function=Function(arguments='{"keyword": "xindoo"}', name='search_google'), type='function'), ChatCompletionMessageToolCall(id='call_Uz8nmwejrMzdz9iVRDCjKC8g', function=Function(arguments='{"keyword": "xindoo"}', name='search_bing'), type='function')])
[{'role': 'user', 'content': "汇总下百度、谷歌、必应三个搜索引擎关于'xindoo'的结果"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_E9tvxYXgM3aYhnMMjF5AbgeW', function=Function(arguments='{"keyword": "xindoo"}', name='search_baidu'), type='function'), ChatCompletionMessageToolCall(id='call_Z9Aw9mIzXKbrbF69LXwATBgT', function=Function(arguments='{"keyword": "xindoo"}', name='se